# Reseau de neuronne

## Importation des librairies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report

2023-01-07 22:08:28.762586: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-07 22:08:29.001795: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-07 22:08:29.001822: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-07 22:08:30.306491: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## I. Lecture des descripteurs

In [2]:
data = pd.read_csv('../../predict/model/output.csv')
data = np.array(data.values)
label_names = data[:,1024]
unique_label = np.unique(label_names)
feature_values = data[:, :1024]

## II. Réseau de neuronne

On commence par séparer les données en deux ensembles: un pour l'apprentissage et l'autre pour l'évaluation du modèle

In [3]:
# split données
X_train, X_test, y_train, y_test = train_test_split(feature_values, label_names, test_size=0.33, random_state=43)

On poursuit en transformant les labels en indice:

In [4]:
# enc labelNames to indices
encName2Ind = preprocessing.LabelEncoder()
encName2Ind.fit(unique_label)
labelIndices_unique = encName2Ind.transform(unique_label)
labelIndices  = encName2Ind.transform(label_names)

# Conversion des noms des labels en indices
labelInd_train = encName2Ind.transform(y_train)
labelInd_test = encName2Ind.transform(y_test)

X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_train = tf.convert_to_tensor(labelInd_train, dtype=tf.float32)
y_test = tf.convert_to_tensor(labelInd_test, dtype=tf.float32)

2023-01-07 22:08:32.949838: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-07 22:08:32.949897: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-07 22:08:32.949943: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nicolas-Lenovo-IdeaPad-C340-14IWL): /proc/driver/nvidia/version does not exist
2023-01-07 22:08:32.952757: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


On normalise les données pour réduire la complexité et donner le même poids à tous les descripteurs.

In [5]:
# Normalisation 
images_mean = np.mean(X_train, axis=0, keepdims=True)
images_std = np.std(X_test, axis=0, keepdims=True) + 1e-7
X_train = (X_train - images_mean) / images_std
X_test = (X_test - images_mean) / images_std

/home/nicolas/Documents/3A_ENSTA/ml_embedded/vultr-tensorflow/lib/python3.8/site-packages/numpy/core/_methods.py:239: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


On crée ensuite le réseau de neuronne.

In [6]:
## creation du reseau de neuronne
model = Sequential(name="Sequential_CNN")
# couche de classification
model.add(Dense(256, activation='relu', input_shape = [1024]))
model.add(Dense(len(unique_label), activation='softmax'))
model.summary()

Model: "Sequential_CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               262400    
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 264,970
Trainable params: 264,970
Non-trainable params: 0
_________________________________________________________________


Apprentissage :

In [7]:
# paramètres
NB_EPOCHS = 40
LEARNING_RATE = 0.1
MOMENTUM = 0.09
DECAY = 0.01

#  Choix de la tâche à effectuer = choix de la fonction de loss/pertes/coûts à minimise
loss_fcn = tf.keras.losses.SparseCategoricalCrossentropy()# sans alias

# compilation ...
model.compile(loss=loss_fcn, optimizer='ADAM', metrics=['accuracy'])
#model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_asbolute_error', 'mean_squared_error'])

#... et entrainement
history = model.fit(X_train, y_train, epochs=NB_EPOCHS)

Epoch 1/40
21/21 [==============================] - 1s 6ms/step - loss: 1.9886 - accuracy: 0.2915
Epoch 2/40
21/21 [==============================] - 0s 6ms/step - loss: 1.5344 - accuracy: 0.4649
Epoch 3/40
21/21 [==============================] - 0s 5ms/step - loss: 1.3527 - accuracy: 0.5157
Epoch 4/40
21/21 [==============================] - 0s 6ms/step - loss: 1.2178 - accuracy: 0.5994
Epoch 5/40
21/21 [==============================] - 0s 6ms/step - loss: 1.1355 - accuracy: 0.6263
Epoch 6/40
21/21 [==============================] - 0s 7ms/step - loss: 1.0550 - accuracy: 0.6726
Epoch 7/40
21/21 [==============================] - 0s 6ms/step - loss: 1.0109 - accuracy: 0.6726
Epoch 8/40
21/21 [==============================] - 0s 6ms/step - loss: 0.9185 - accuracy: 0.7309
Epoch 9/40
21/21 [==============================] - 0s 7ms/step - loss: 0.8664 - accuracy: 0.7309
Epoch 10/40
21/21 [==============================] - 0s 5ms/step - loss: 0.8373 - accuracy: 0.7399
Epoch 11/40
21/21 [

## III. Evaluation du modèle

In [13]:
#Confution Matrix and Classification Report
Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)
print()
print()
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))
print()
print()
print('Classification Report')
print(classification_report(labelInd_test, y_pred, target_names=unique_label))

11/11 [==============================] - 0s 4ms/step


Confusion Matrix
[[15  0  2  1  1  1  1  0  2  7]
 [ 1 28  0  0  0  1  0  0  0  0]
 [ 2  1 10  3  3  7  0  0  5  6]
 [ 2  1  0 21  1  1  5  2  4  5]
 [ 0  0  0  3 23  0  2  4  3  1]
 [ 0 11  0  1  0 13  3  0  0  2]
 [ 1  1  0  0  1  0 27  0  1  1]
 [ 0  1  0  2  0  1  0 28  0  0]
 [ 1  1  2  2  4  1  3  2 16  2]
 [ 1  2  0  4  0  3  5  0  2 10]]


Classification Report
              precision    recall  f1-score   support

       blues       0.65      0.50      0.57        30
   classical       0.61      0.93      0.74        30
     country       0.71      0.27      0.39        37
       disco       0.57      0.50      0.53        42
      hiphop       0.70      0.64      0.67        36
        jazz       0.46      0.43      0.45        30
       metal       0.59      0.84      0.69        32
         pop       0.78      0.88      0.82        32
      reggae       0.48      0.47      0.48        34
        rock       0.29      0.3

## IV. Enregistrement du modèle sous tensorflow-lite

In [9]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('../../predict/model/model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpaznyumee/assets


INFO:tensorflow:Assets written to: /tmp/tmpaznyumee/assets
2023-01-07 22:08:41.460578: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-01-07 22:08:41.460609: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-01-07 22:08:41.461252: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpaznyumee
2023-01-07 22:08:41.462234: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-01-07 22:08:41.462252: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpaznyumee
2023-01-07 22:08:41.465870: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-01-07 22:08:41.467475: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-01-07 22:08:41.514596: I tensorflow/cc/saved_model/loader.cc:213] Running initializatio

enregistrement des données d'évaluation

In [40]:
txt = ""
txt += "#ifndef TEST_DATASET_H\n#define TEST_DATASET_H\n\n"
txt += "#define N_TEST_EXAMPLES " + str(len(X_test)) + "\n"
txt += "#define N_FEATURES " + str(len(y_test)) + "\n"
txt += "#define N_CLASSES 10\n\n"

txt += "float test_acc=" + str(model.evaluate(X_test, y_test)[1]) +";\n\n"

txt += "float X_test[N_TEST_EXAMPLES][N_FEATURES] = {\n"
for i in range(len(X_test)-1):
    # print(X_test[i].numpy())
    txt += "{"
    for j in range(len(y_test)-1):
        txt += str(X_test[i,j].numpy()) +","
    txt += str(X_test[i,len(y_test)-1].numpy()) + "},\n"
txt += "{"
for j in range(len(y_test)-1):
    txt += str(X_test[len(X_test)-1,j].numpy()) +","
txt += str(X_test[len(X_test)-1,len(y_test)-1].numpy()) + "}};\n\n"

txt += "int y_test[N_FEATURES]={"
for i in range(len(y_test)-1):
    txt += str(y_test[i].numpy()) +","
txt += str(y_test[len(y_test)-1].numpy()) +"};\n\n"

txt += "#endif"

# écriture du code dans le fichier
fichier = open("../../predict/model/test_dataset.h", "w")
fichier.write(txt)
fichier.close()

11/11 [==============================] - 0s 3ms/step - loss: 2.2261 - accuracy: 0.5788
